<h1>Chapter 1 - Introduction to Language Models</h1>
<i>Exploring the exciting field of Language AI</i>


<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter01/Chapter%201%20-%20Introduction%20to%20Language%20Models.ipynb)

---

This notebook is for Chapter 1 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---

In [1]:
# %%capture
# !pip install transformers>=4.40.1 accelerate>=0.27.2

In [2]:
!nvidia-smi

Mon Mar 10 09:47:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
os.environ["HF_HOME"] = "/openbayes/home/huggingface"

In [4]:
!echo $HF_HOME

/openbayes/home/huggingface


# Phi-3

The first step is to load our model onto the GPU for faster inference. Note that we load the model and tokenizer separately (although that isn't always necessary).

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# 加载模型和 tokenizer （第一次加载的时候会进行下载）
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
# 查看 模型结构是什么？
print(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [6]:
# 这里有一个 Special token, 打印看一下是什么token呢？
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
# tokenizer.all_special_tokens
tokenizer.special_tokens_map

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

1.2 基础使用方式

加载模型之后可以怎么使用呢？可以直接使用原始的 model 和 tokenizer 进行推理；

step1: 加载模型

step2: 构建 prompt 和 tokenizer

step3: 推理和解码

刚刚已经加载过模型了，所以直接进行 step2

In [7]:
prompt = "讲一个猫有关的笑话？"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

#  想一下诗变成什么格式
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

print(text)
print("====" * 10)
print(model_inputs)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
讲一个猫有关的笑话？<|im_end|>
<|im_start|>assistant

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,  99526,  46944, 100472,
         101063,   9370, 109959,  11319, 151645,    198, 151644,  77091,    198]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [8]:
# step3: 推理和解码
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

在一个名叫“喵星人”的猫咪世界里，有这样一则趣事：

有一天，一只名叫“小猫”的猫咪去寻找它的尾巴。它在街上走着，突然发现了一只大狗正对着马路。小狗非常生气，大声吼叫着说：“喂！别来这儿找我！”小猫很无辜地问：“为什么？”小狗回答：“因为我的尾巴现在被你抓走了。”小猫听后，立刻意识到自己的错误，赶紧向小狗道歉并表示愿意帮助它找回尾巴。

从此以后，“喵星人”变成了一个乐于助人的猫咪，大家都喜欢和它玩捉迷藏游戏。这个故事告诉我们，遇到困难时，我们应当学会宽容和理解，以一颗善良的心去对待他人，让这个世界变得更加美好。


1.3 使用 transformers 的 pipeline 简化流程

Although we can now use the model and tokenizer directly, it's much easier to wrap it in a `pipeline` object:

In [9]:
from transformers import pipeline
# step1: 生成 pipeline
# return_full_text=False 表示只返回新生成的文本，不包含输入的prompt
# return_full_text=True 则会返回完整文本，包含输入的prompt和生成的新文本
# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,  # 只返回新生成的文本部分
    max_new_tokens=500,
    do_sample=False
)
# step2: 构建 prompt
messages = [
    {"role": "user", "content": "写一个和猫有关的笑话."}
]

# step3，输出并解码
output = generator(messages)
print(output[0]["generated_text"])

Device set to use cuda
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


好的，以下是一个关于猫的笑话：

有一天，一只猫在森林里迷路了。它四处张望，但什么也看不见。突然，它看到了一棵树上挂着一个牌子，上面写着“欢迎光临”。猫好奇地走过去，发现牌子后面有一个小洞。

猫小心翼翼地走进洞穴，里面有一只小老鼠正在吃着食物。猫对老鼠说：“你好，我是来自森林的小动物。”老鼠回答道：“我叫米奇，很高兴见到你。”

猫对米奇说：“我也很高兴遇见你，但我需要一些帮助。我的主人告诉我，如果你能找到一块石头，就能回到森林中去。你能帮我吗？”

米奇想了想，然后从洞穴里拿出了一块大石头。猫感激地说：“谢谢你，米奇！我会好好珍惜这块石头的。”

从此以后，每当猫迷路时，就会想起那只小老鼠和它的故事，而米奇也会一直陪伴在它身边。


In [10]:
output

[{'generated_text': '好的，以下是一个关于猫的笑话：\n\n有一天，一只猫在森林里迷路了。它四处张望，但什么也看不见。突然，它看到了一棵树上挂着一个牌子，上面写着“欢迎光临”。猫好奇地走过去，发现牌子后面有一个小洞。\n\n猫小心翼翼地走进洞穴，里面有一只小老鼠正在吃着食物。猫对老鼠说：“你好，我是来自森林的小动物。”老鼠回答道：“我叫米奇，很高兴见到你。”\n\n猫对米奇说：“我也很高兴遇见你，但我需要一些帮助。我的主人告诉我，如果你能找到一块石头，就能回到森林中去。你能帮我吗？”\n\n米奇想了想，然后从洞穴里拿出了一块大石头。猫感激地说：“谢谢你，米奇！我会好好珍惜这块石头的。”\n\n从此以后，每当猫迷路时，就会想起那只小老鼠和它的故事，而米奇也会一直陪伴在它身边。'}]

Finally, we create our prompt as a user and give it to the model:

In [ ]:
# The prompt (user input / query)
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# Generate output
output = generator(messages)
print(output[0]["generated_text"])

 Why did the chicken join the band? Because it had the drumsticks!
